In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import leaguedashteamstats


fig_DIR = "../figs/team_ratings/"
box_DIR = "../data/box/"

In [ ]:
cols1 = ['TEAM_NAME', 'GP', 'W', 'L', 'W_PCT',
       'OFF_RATING', 'DEF_RATING', 'NET_RATING', 'EFG_PCT', 'TS_PCT', 'E_PACE', 'PACE', 'POSS',
       "W_RANK", "L_RANK",
       'W_PCT_RANK', 'OFF_RATING_RANK', 'DEF_RATING_RANK',
       'NET_RATING_RANK',]

In [ ]:
dfa = []
for year in range(2014,2024):
    stats = leaguedashteamstats.LeagueDashTeamStats(
        measure_type_detailed_defense="Advanced",
        season = get_ss(year)
    )
    df1 = stats.get_data_frames()[0]
    df1 = df1.sort_values("W_PCT",ascending=False).reset_index(drop=True)
    time.sleep(0.6)
    stats = leaguedashteamstats.LeagueDashTeamStats(
        measure_type_detailed_defense="Advanced",
        season_type_all_star="Playoffs",
        season=get_ss(year)
    )
    df2 = stats.get_data_frames()[0]
    df2 = df2.sort_values("W",ascending=False).reset_index(drop=True)
    df1 = df1[cols1]
    df2 = df2[cols1]
    df3 = pd.merge(df2,df1,on=["TEAM_NAME"],how="left")
    df3["season"] = year + 1
    
    time.sleep(0.6)
    stats = leaguegamelog.LeagueGameLog(
        player_or_team_abbreviation="T",
        season=get_ss(year),
        season_type_all_star="Playoffs",
    )
    df4 = stats.get_data_frames()[0]
    df4["Round"] = df4["GAME_ID"].str[-3]
    df4["Win"] = df4["WL"] == "W"
    df4["Loss"] = df4["WL"] == "L"
    df5 = df4.groupby(["TEAM_NAME","Round"])[["Win","Loss"]].sum().reset_index()
    df6 = df5.groupby(["TEAM_NAME"])[["Round"]].count().reset_index()
    df7 = pd.merge(df3,df6,on="TEAM_NAME",how="left")
    dfa.append(df7)
    time.sleep(0.6)
df8 = pd.concat(dfa)
df8 = df8.rename(columns={"Round":"Round_x"})

In [ ]:
df8

In [ ]:
qus = ["Round_x > 0","Round_x > 1","Round_x > 2","Round_x > 3","Round_x > 1 & W_RANK_x == 1"]
scenario = ["Playoffs","2nd Round","Conf Finals","Finals","Championship"]

In [ ]:
dfb = []
for q,s in zip(qus,scenario):
    df9 = df8.query(q)
    df9["Scenario"] = s
    dfb.append(df9)
df10 = pd.concat(dfb)

In [ ]:
df11 = df10[["TEAM_NAME","season","Scenario"]]